## The steps of this code are as follows:
1. Import the RTKLIB corrected RINEX drone data
1. Import the Timestamp.MRK file created from the UAV
1. Create a calculation dataframe to correct imagery locations based on the corrected RTKLIB file
1. Create a new export file with updated Lat, Lon, and elevation and associated errors
1. Read the original exif data from the images and append that to your export file
1. Subtract the observed snow depth at the RTK site from each elevation observation
1. Export a final .csv with the updated lat, long, elevation and pitch, roll, yaw 

In [1]:
# Import numerical tools
import numpy as np
# Import pyplot for plotting
import matplotlib.pyplot as plt
#Import pandas for reading in and managing data
import pandas as pd
import math
# Magic function to make matplotlib inlineSet the filename for the code used for your imagery collection. This is the first 7 digits when you download imagery. Set the date for your flight collections.; other style specs must come AFTER
%matplotlib inline
%config InlineBackend.figure_formats = {'svg',}
#Comment the above line and uncomment the line below if svg graphics are not working in your browser.
#%config InlineBackend.figure_formats = {'png', 'retina'}
import os
import glob
import os.path

## Set the filename for the code used for your imagery collection. This is the first 7 digits when you download imagery. Set the date for your flight collections.

In [2]:
# Set the date of the flight to search for the working directory for each time this code is run.
date = '20210624_SnowOff'

In [3]:
# Set your working directory
# Hourglass processing work should be in the folder 'HG_2022_PPK_Processing' and should be parsed out by date
# Since each date will have its own folder, the date in the working directory should be the only thing changed in
# this form.
os.chdir('/Users/f67f911/Desktop/HG_PPK/' + date)

## Step 1: Read in the already corrected RINEX file from the drone. This file should always be saved in your directory as filename_Rinex.csv so that the following code works. This is from the drone capturing positional location throughout the entirety of the flight, so there will be many more rows of data than there are pictures taken during that flight. 

In [4]:
# Designate the file paths for the raw and imagery files within the processing folder
raw_files_path = 'Raw_files/'
imagery_path = 'Imagery/'

In [5]:
# Capture all of the Rinex Files
rinex_files = glob.glob(raw_files_path + '/*.csv')
timestamp_files = glob.glob(raw_files_path + '/*.MRK')

In [6]:
# Sort the files and check to make sure each has been populated correctly
rinex_files = sorted(rinex_files)
rinex_files

['Raw_files/101_0222_Rinex.csv',
 'Raw_files/101_0223_Rinex.csv',
 'Raw_files/101_0224_Rinex.csv',
 'Raw_files/101_0225_Rinex.csv']

In [7]:
# Sort and check the timestamp files
timestamp_files = sorted(timestamp_files)
timestamp_files

['Raw_files/101_0222_Timestamp.MRK',
 'Raw_files/101_0223_Timestamp.MRK',
 'Raw_files/101_0224_Timestamp.MRK',
 'Raw_files/101_0225_Timestamp.MRK']

In [8]:
# Create an empty list to hold the data from each .csv file
rinex_dfholder = []
# For each file in the rinex files variable, read the csv file and append to the dfholder
for i in rinex_files:
        # Create a new dataframe for each file
        df = pd.read_csv(i, index_col = None, header = 0)
        df = pd.DataFrame(df)
        # Add flight number data to differentiate what flight each data was collected with
        flight = os.path.basename(i)
        flight = os.path.splitext(flight)[0]
        # Replace the 'Rinex' descriptor with blank characters
        flight = flight.replace('_Rinex', '')
        # Create a new column in the dataframe to hold the flight number values
        df['flight'] = flight
        # Append the list of df_holder with all of the values for the individual dataframes
        rinex_dfholder.append(df)
# Concat all of the data into a final dataframe
RTKLIB_record = pd.concat(rinex_dfholder, axis = 0, ignore_index = True)
RTKLIB_record

,%,GPST,latitude(deg),longitude(deg),height(m),Q,ns,sdn(m),sde(m),sdu(m),sdne(m),sdeu(m),sdun(m),age(s),ratio,flight
0,2163,401931.0,45.834855,-110.934317,2316.1361,1,6,0.0098,0.0098,0.0204,0.0029,0.0066,0.0032,-9.0,989.4,101_0222
1,2163,401931.2,45.834855,-110.934317,2316.1324,1,6,0.0097,0.0096,0.0202,0.0028,0.0065,0.0030,-8.8,983.9,101_0222
2,2163,401931.4,45.834856,-110.934317,2316.1294,1,6,0.0096,0.0095,0.0199,0.0028,0.0064,0.0029,-8.6,979.3,101_0222
3,2163,401931.6,45.834856,-110.934317,2316.1238,1,6,0.0094,0.0093,0.0196,0.0028,0.0063,0.0027,-8.4,974.3,101_0222
4,2163,401931.8,45.834856,-110.934317,2316.1276,1,6,0.0093,0.0091,0.0193,0.0028,0.0061,0.0025,-8.2,968.9,101_0222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10865,2163,405696.4,45.835128,-110.934099,2296.9443,1,7,0.0073,0.0060,0.0192,-0.0021,-0.0038,0.0031,6.4,999.9,101_0225
10866,2163,405696.6,45.835125,-110.934107,2296.9397,1,7,0.0074,0.0061,0.0192,-0.0021,-0.0038,0.0031,6.6,999.9,101_0225
10867,2163,405696.8,45.835123,-110.934114,2296.9488,1,7,0.0074,0.0061,0.0179,-0.0021,-0.0035,0.0028,6.8,999.9,101_0225
10868,2163,405697.0,45.835121,-110.934120,2296.9616,1,7,0.0063,0.0054,0.0129,-0.0016,-0.0024,0.0019,7.0,999.9,101_0225


## Step 2: Read in the timestamp data as is from the UAV. 

In [9]:
# Create an empty list to hold the data from each .csv file
timestamp_dfholder = []
# For each file in the rinex files variable, read the csv file and append to the dfholder
for j in timestamp_files:
        # Create a new dataframe for each file
        df = pd.read_table(j, index_col = None, header = None)
        df = pd.DataFrame(df)
        # Add flight number data to differentiate what flight each data was collected with
        flight = os.path.basename(j)
        flight = os.path.splitext(flight)[0]
        # Replace the 'Rinex' descriptor with blank characters
        flight = flight.replace('_Timestamp', '')
        # Create a new column in the dataframe to hold the flight number values
        df['flight'] = flight
        timestamp_dfholder.append(df)
# Concat all of the data into a final dataframe
timestamp_record = pd.concat(timestamp_dfholder, axis = 0, ignore_index = True)
timestamp_record.columns = ['Photo', 'GPS_Date','% GPST','Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Lat','Lon','Height_m','std_North_m, std_East_m, std_Ele_m','RTK_status_flag','flight']
timestamp_record.head()

,Photo,GPS_Date,% GPST,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Lat,Lon,Height_m,"std_North_m, std_East_m, std_Ele_m",RTK_status_flag,flight
0,1,401937.285994,[2163],"19,N","2,E","193,V","45.83487235,Lat","-110.93434036,Lon","2314.668,Ellh","0.015066, 0.018024, 0.035783","50,Q",101_0222
1,2,401941.449543,[2163],"-8,N","-24,E","193,V","45.83480398,Lat","-110.93446262,Lon","2321.321,Ellh","0.015246, 0.018220, 0.034469","50,Q",101_0222
2,3,401944.974985,[2163],"-7,N","-24,E","193,V","45.83474465,Lat","-110.93457321,Lon","2327.427,Ellh","0.015726, 0.018616, 0.034034","50,Q",101_0222
3,4,401948.524124,[2163],"-12,N","-29,E","192,V","45.83468431,Lat","-110.93468434,Lon","2333.503,Ellh","0.015270, 0.018342, 0.033108","50,Q",101_0222
4,5,401952.052273,[2163],"-9,N","-29,E","192,V","45.83462552,Lat","-110.93479366,Lon","2339.543,Ellh","0.015523, 0.018539, 0.033029","50,Q",101_0222


In [10]:
# Create a new dataframe to hold the timestamp record data in case we mess up in the future
timestamp = timestamp_record

### Note status flag values - 0: no positioning; 16: single-point positioning mode; 34: RTK floating solution; 50: RTK fixed solution. When flag of a photo is not equal to 50, it is recommended that you should not use that image in further processing.

## Clean the timestamp file to convert non-numeric text in columns to numeric

## When looking at our columns, we can see that there are many numbers followed by letters. We need to get rid of those numbers in order to continue with our analysis. 
### The following code removes any non-numeric value from the columns in the dataframe.

In [11]:
# For each column in the timestamp dataframe
for col in timestamp:
    if col == 'flight':
        timestamp[col] = timestamp[col]
    # If the column name is '% GPST'
    elif col == '% GPST':
        # Remove the brackets around the value and change the data to type int
        timestamp[col] = timestamp[col].str.replace('[','', regex = True).str.replace(']','', regex = True).astype(int)
    # Or if the column is equal to the standard deviation column
    elif col == 'std_North_m, std_East_m, std_Ele_m':
        # Leave it alone
        timestamp[col] = timestamp[col]
    # If the data type of the column is an object
    elif timestamp.dtypes[col] == object:
        # Replace non-numeric values with a blank value
        timestamp[col] = timestamp[col].str.replace(r'\,\D+', '', regex = True).astype(float)
# View the data to make sure it has been changed correctly
timestamp.head()

,Photo,GPS_Date,% GPST,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Lat,Lon,Height_m,"std_North_m, std_East_m, std_Ele_m",RTK_status_flag,flight
0,1,401937.285994,2163,19.0,2.0,193.0,45.834872,-110.934340,2314.668,"0.015066, 0.018024, 0.035783",50.0,101_0222
1,2,401941.449543,2163,-8.0,-24.0,193.0,45.834804,-110.934463,2321.321,"0.015246, 0.018220, 0.034469",50.0,101_0222
2,3,401944.974985,2163,-7.0,-24.0,193.0,45.834745,-110.934573,2327.427,"0.015726, 0.018616, 0.034034",50.0,101_0222
3,4,401948.524124,2163,-12.0,-29.0,192.0,45.834684,-110.934684,2333.503,"0.015270, 0.018342, 0.033108",50.0,101_0222
4,5,401952.052273,2163,-9.0,-29.0,192.0,45.834626,-110.934794,2339.543,"0.015523, 0.018539, 0.033029",50.0,101_0222


### Calculate camera specific positions. 
Step one is to create a calculations spreadsheet.

In [12]:
# Create a new dataframe with set columns for our future calculations
calc = pd.DataFrame(columns = ['Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm','Closest_Loc_ID',
                    'Timestamp_of_Closest','Closest_Lat','Closest_Lon','Closest_El','2nd_Closest_Loc_ID',
                    'Timestamp_of_2nd_Closest','2nd_Closest_Lat','2nd_Closest_Lon','2nd_Closest_El',
                    'Percent_diff_between_timestamps','Interpolated_Lat','Interpolated_Lon','Interpolated_El',
                    'Lat_Diff_deg','Lon_Diff_deg','El_diff_m','New_Lat','New_Lon','New_El']).astype(int)

### Calculate the values and input into the calc dataframe. This output will include a new latitude, longitude, and elevation.

In [13]:
# Populate the new dataframe with values from the timestamp data
calc['Northing_diff_mm'] = timestamp['Northing_diff_mm']
calc['Easting_diff_mm'] = timestamp['Easting_diff_mm']
calc['Elevation_diff_mm'] = timestamp['Elevation_diff_mm']
# Read in the data to make sure these columns have populated
calc.head()

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Percent_diff_between_timestamps,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El
0,19.0,2.0,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-8.0,-24.0,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-7.0,-24.0,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-12.0,-29.0,192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-9.0,-29.0,192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 3: Convert the latitude difference into degrees.

In [14]:
# First lets add a column in the RTKLIB data to populate out the timestamps.
# Set a record in case we mess up this data
RTKLIB = RTKLIB_record
RTKLIB.index.name = 'ID'
RTKLIB.reset_index(inplace = True)
RTKLIB.head()

,ID,%,GPST,latitude(deg),longitude(deg),height(m),Q,ns,sdn(m),sde(m),sdu(m),sdne(m),sdeu(m),sdun(m),age(s),ratio,flight
0,0,2163,401931.0,45.834855,-110.934317,2316.1361,1,6,0.0098,0.0098,0.0204,0.0029,0.0066,0.0032,-9.0,989.4,101_0222
1,1,2163,401931.2,45.834855,-110.934317,2316.1324,1,6,0.0097,0.0096,0.0202,0.0028,0.0065,0.0030,-8.8,983.9,101_0222
2,2,2163,401931.4,45.834856,-110.934317,2316.1294,1,6,0.0096,0.0095,0.0199,0.0028,0.0064,0.0029,-8.6,979.3,101_0222
3,3,2163,401931.6,45.834856,-110.934317,2316.1238,1,6,0.0094,0.0093,0.0196,0.0028,0.0063,0.0027,-8.4,974.3,101_0222
4,4,2163,401931.8,45.834856,-110.934317,2316.1276,1,6,0.0093,0.0091,0.0193,0.0028,0.0061,0.0025,-8.2,968.9,101_0222


In [15]:
# First, we need to create constant values with numbers used in further calculations
# as conversion factors
# 1 degree latitude in meters
deg_lat_m = 111111
# The latitude used. Use the first latitude value from the RTKLIB values
Lon_used = RTKLIB.iloc[0,RTKLIB.columns.get_loc('longitude(deg)')]
# 1 degree longitude in meters
deg_lon_m = abs(math.cos(np.deg2rad(Lon_used))*111111)

In [16]:
# This code calculates the latitude difference in degrees for this dataset. 
calc['Lat_Diff_deg'] = calc['Northing_diff_mm']/1000/deg_lat_m
# The longitude difference in degrees
calc['Lon_Diff_deg'] = calc['Easting_diff_mm']/1000/deg_lon_m
# The elevation difference in meters
calc['El_diff_m'] = calc['Elevation_diff_mm']/1000
# Call the head of the dataframe to make sure these calculations were done correctly
calc.head()

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Percent_diff_between_timestamps,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El
0,19.0,2.0,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.710002e-07,5.037824e-08,0.193,NaN,NaN,NaN
1,-8.0,-24.0,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-7.200007e-08,-6.045389e-07,0.193,NaN,NaN,NaN
2,-7.0,-24.0,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-6.300006e-08,-6.045389e-07,0.193,NaN,NaN,NaN
3,-12.0,-29.0,192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.080001e-07,-7.304845e-07,0.192,NaN,NaN,NaN
4,-9.0,-29.0,192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-8.100008e-08,-7.304845e-07,0.192,NaN,NaN,NaN


### Calculate the closest latitude. To do this, we first need to calculate the timestamp that is closest.

In [17]:
# Create a merged dataframe that merges the two dataframes on the closest time values
merge = pd.merge_asof(timestamp, RTKLIB.sort_values('GPST'), left_on = 'GPS_Date', right_on = 'GPST', 
                      direction = 'nearest')
# View the new column names to be able to populate the calc dataframe
merge.columns

Index(['Photo', 'GPS_Date', '% GPST', 'Northing_diff_mm', 'Easting_diff_mm',
       'Elevation_diff_mm', 'Lat', 'Lon', 'Height_m',
       'std_North_m, std_East_m, std_Ele_m', 'RTK_status_flag', 'flight_x',
       'ID', '%', 'GPST', 'latitude(deg)', 'longitude(deg)', 'height(m)', 'Q',
       'ns', 'sdn(m)', 'sde(m)', 'sdu(m)', 'sdne(m)', 'sdeu(m)', 'sdun(m)',
       'age(s)', 'ratio', 'flight_y'],
      dtype='object')

In [18]:
# Populate columns of the calc dataframe with data from the merged dataframe
calc['Closest_Loc_ID'] = merge['ID']
calc['Timestamp_of_Closest'] = merge['GPST']
calc['Closest_Lat'] = merge['latitude(deg)']
calc['Closest_Lon'] = merge['longitude(deg)']
calc['Closest_El'] = merge['height(m)']
# Add in the flight identifier column 
calc['flight'] = merge['flight_y']
# View to make sure it has been read in correctly
calc.head()

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El,flight
0,19.0,2.0,193.0,31,401937.2,45.834855,-110.934318,2316.1912,NaN,NaN,...,NaN,NaN,NaN,1.710002e-07,5.037824e-08,0.193,NaN,NaN,NaN,101_0222
1,-8.0,-24.0,193.0,52,401941.4,45.834788,-110.934439,2322.8243,NaN,NaN,...,NaN,NaN,NaN,-7.200007e-08,-6.045389e-07,0.193,NaN,NaN,NaN,101_0222
2,-7.0,-24.0,193.0,70,401945.0,45.834727,-110.934552,2329.0167,NaN,NaN,...,NaN,NaN,NaN,-6.300006e-08,-6.045389e-07,0.193,NaN,NaN,NaN,101_0222
3,-12.0,-29.0,192.0,88,401948.6,45.834666,-110.934664,2335.1888,NaN,NaN,...,NaN,NaN,NaN,-1.080001e-07,-7.304845e-07,0.192,NaN,NaN,NaN,101_0222
4,-9.0,-29.0,192.0,105,401952.0,45.834609,-110.934770,2341.0124,NaN,NaN,...,NaN,NaN,NaN,-8.100008e-08,-7.304845e-07,0.192,NaN,NaN,NaN,101_0222


In [19]:
# Now, we should populate the second closest values
# Add one to the closest location ID
calc['2nd_Closest_Loc_ID'] = calc['Closest_Loc_ID'] + 1
# And now populate the data from the RTKLIB file that matches that ID value
calc['Timestamp_of_2nd_Closest'] =  calc['2nd_Closest_Loc_ID'].map(RTKLIB['GPST'])
calc['2nd_Closest_Lat'] = calc['2nd_Closest_Loc_ID'].map(RTKLIB['latitude(deg)'])
calc['2nd_Closest_Lon'] = calc['2nd_Closest_Loc_ID'].map(RTKLIB['longitude(deg)'])
calc['2nd_Closest_El'] = calc['2nd_Closest_Loc_ID'].map(RTKLIB['height(m)'])
# View the dataframe to make sure it was populated correctly
calc.head()                                                 

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El,flight
0,19.0,2.0,193.0,31,401937.2,45.834855,-110.934318,2316.1912,32,401937.4,...,NaN,NaN,NaN,1.710002e-07,5.037824e-08,0.193,NaN,NaN,NaN,101_0222
1,-8.0,-24.0,193.0,52,401941.4,45.834788,-110.934439,2322.8243,53,401941.6,...,NaN,NaN,NaN,-7.200007e-08,-6.045389e-07,0.193,NaN,NaN,NaN,101_0222
2,-7.0,-24.0,193.0,70,401945.0,45.834727,-110.934552,2329.0167,71,401945.2,...,NaN,NaN,NaN,-6.300006e-08,-6.045389e-07,0.193,NaN,NaN,NaN,101_0222
3,-12.0,-29.0,192.0,88,401948.6,45.834666,-110.934664,2335.1888,89,401948.8,...,NaN,NaN,NaN,-1.080001e-07,-7.304845e-07,0.192,NaN,NaN,NaN,101_0222
4,-9.0,-29.0,192.0,105,401952.0,45.834609,-110.934770,2341.0124,106,401952.2,...,NaN,NaN,NaN,-8.100008e-08,-7.304845e-07,0.192,NaN,NaN,NaN,101_0222


In [20]:
# Calculate the percent difference between the timestamp values
calc['Percent_diff_between_timestamps'] = (timestamp['GPS_Date'] - calc['Timestamp_of_Closest'])/(calc['Timestamp_of_2nd_Closest'] - calc['Timestamp_of_Closest'])

In [21]:
# Calculate the interpolated latitude, longitude, and elevation based on the differences in timestamp data
calc['Interpolated_Lat'] = ((calc.Closest_Lat)*(1-calc.Percent_diff_between_timestamps))+(calc['2nd_Closest_Lat'])*(calc.Percent_diff_between_timestamps)
calc['Interpolated_Lon'] = ((calc.Closest_Lon)*(1-calc.Percent_diff_between_timestamps))+(calc['2nd_Closest_Lon'])*(calc.Percent_diff_between_timestamps)
calc['Interpolated_El'] = ((calc.Closest_El)*(1-calc.Percent_diff_between_timestamps))+(calc['2nd_Closest_El'])*(calc.Percent_diff_between_timestamps)
# View to make sure it has been populated correctly
calc

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El,flight
0,19.0,2.0,193.0,31,401937.2,45.834855,-110.934318,2316.1912,32,401937.4,...,45.834855,-110.934319,2316.218030,1.710002e-07,5.037824e-08,0.193,NaN,NaN,NaN,101_0222
1,-8.0,-24.0,193.0,52,401941.4,45.834788,-110.934439,2322.8243,53,401941.6,...,45.834787,-110.934440,2322.910431,-7.200007e-08,-6.045389e-07,0.193,NaN,NaN,NaN,101_0222
2,-7.0,-24.0,193.0,70,401945.0,45.834727,-110.934552,2329.0167,71,401945.2,...,45.834727,-110.934551,2328.974425,-6.300006e-08,-6.045389e-07,0.193,NaN,NaN,NaN,101_0222
3,-12.0,-29.0,192.0,88,401948.6,45.834666,-110.934664,2335.1888,89,401948.8,...,45.834667,-110.934662,2335.057800,-1.080001e-07,-7.304845e-07,0.192,NaN,NaN,NaN,101_0222
4,-9.0,-29.0,192.0,105,401952.0,45.834609,-110.934770,2341.0124,106,401952.2,...,45.834608,-110.934771,2341.102466,-8.100008e-08,-7.304845e-07,0.192,NaN,NaN,NaN,101_0222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,-9.0,-18.0,193.0,10778,405679.0,45.835394,-110.933286,2297.2340,10779,405679.2,...,45.835394,-110.933286,2297.234003,-8.100008e-08,-4.534042e-07,0.193,NaN,NaN,NaN,101_0225
559,0.0,-33.0,192.0,10800,405683.4,45.835328,-110.933495,2297.3130,10801,405683.6,...,45.835329,-110.933492,2297.310624,0.000000e+00,-8.312410e-07,0.192,NaN,NaN,NaN,101_0225
560,4.0,-18.0,194.0,10822,405687.8,45.835261,-110.933699,2297.2775,10823,405688.0,...,45.835263,-110.933695,2297.277755,3.600004e-08,-4.534042e-07,0.194,NaN,NaN,NaN,101_0225
561,4.0,-26.0,193.0,10843,405692.0,45.835197,-110.933896,2297.2595,10844,405692.2,...,45.835196,-110.933897,2297.259760,3.600004e-08,-6.549171e-07,0.193,NaN,NaN,NaN,101_0225


In [22]:
# Calculate the new latitude, longitude, and elevation
calc['New_Lat'] = calc['Interpolated_Lat'] + calc['Lat_Diff_deg']
calc['New_Lon'] = calc['Interpolated_Lon'] + calc['Lon_Diff_deg']
calc['New_El'] = calc['Interpolated_El'] + calc['El_diff_m']
calc

,Northing_diff_mm,Easting_diff_mm,Elevation_diff_mm,Closest_Loc_ID,Timestamp_of_Closest,Closest_Lat,Closest_Lon,Closest_El,2nd_Closest_Loc_ID,Timestamp_of_2nd_Closest,...,Interpolated_Lat,Interpolated_Lon,Interpolated_El,Lat_Diff_deg,Lon_Diff_deg,El_diff_m,New_Lat,New_Lon,New_El,flight
0,19.0,2.0,193.0,31,401937.2,45.834855,-110.934318,2316.1912,32,401937.4,...,45.834855,-110.934319,2316.218030,1.710002e-07,5.037824e-08,0.193,45.834855,-110.934319,2316.411030,101_0222
1,-8.0,-24.0,193.0,52,401941.4,45.834788,-110.934439,2322.8243,53,401941.6,...,45.834787,-110.934440,2322.910431,-7.200007e-08,-6.045389e-07,0.193,45.834787,-110.934441,2323.103431,101_0222
2,-7.0,-24.0,193.0,70,401945.0,45.834727,-110.934552,2329.0167,71,401945.2,...,45.834727,-110.934551,2328.974425,-6.300006e-08,-6.045389e-07,0.193,45.834727,-110.934552,2329.167425,101_0222
3,-12.0,-29.0,192.0,88,401948.6,45.834666,-110.934664,2335.1888,89,401948.8,...,45.834667,-110.934662,2335.057800,-1.080001e-07,-7.304845e-07,0.192,45.834667,-110.934663,2335.249800,101_0222
4,-9.0,-29.0,192.0,105,401952.0,45.834609,-110.934770,2341.0124,106,401952.2,...,45.834608,-110.934771,2341.102466,-8.100008e-08,-7.304845e-07,0.192,45.834608,-110.934772,2341.294466,101_0222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,-9.0,-18.0,193.0,10778,405679.0,45.835394,-110.933286,2297.2340,10779,405679.2,...,45.835394,-110.933286,2297.234003,-8.100008e-08,-4.534042e-07,0.193,45.835393,-110.933286,2297.427003,101_0225
559,0.0,-33.0,192.0,10800,405683.4,45.835328,-110.933495,2297.3130,10801,405683.6,...,45.835329,-110.933492,2297.310624,0.000000e+00,-8.312410e-07,0.192,45.835329,-110.933493,2297.502624,101_0225
560,4.0,-18.0,194.0,10822,405687.8,45.835261,-110.933699,2297.2775,10823,405688.0,...,45.835263,-110.933695,2297.277755,3.600004e-08,-4.534042e-07,0.194,45.835263,-110.933696,2297.471755,101_0225
561,4.0,-26.0,193.0,10843,405692.0,45.835197,-110.933896,2297.2595,10844,405692.2,...,45.835196,-110.933897,2297.259760,3.600004e-08,-6.549171e-07,0.193,45.835197,-110.933897,2297.452760,101_0225


In [23]:
# Create an export dataframe that contains the new locational data
location_export = calc[['New_Lat','New_Lon','New_El','Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm']]
# Reset the index to create a column to hold image ID data
location_export = location_export.reset_index()
# Rename the index column with a descriptive name and then incremently increase the values to match the collected imagery
location_export = location_export.rename(columns = {'index':'FlightID'})
# location_export['PhotoID'] = RTKLIB['flight']
location_export['FlightID'] = calc['flight'].astype(int)
location_export['Lat_diff_m'] = location_export['Northing_diff_mm']/1000
location_export['Lon_diff_m'] = location_export['Easting_diff_mm']/1000
location_export['El_diff_m'] = location_export['Elevation_diff_mm']/1000
location_export = location_export.drop(columns = ['Northing_diff_mm','Easting_diff_mm','Elevation_diff_mm'])
# View the data to make sure that it has been read in correctly.                                          
location_export

,FlightID,New_Lat,New_Lon,New_El,Lat_diff_m,Lon_diff_m,El_diff_m
0,1010222,45.834855,-110.934319,2316.411030,0.019,0.002,0.193
1,1010222,45.834787,-110.934441,2323.103431,-0.008,-0.024,0.193
2,1010222,45.834727,-110.934552,2329.167425,-0.007,-0.024,0.193
3,1010222,45.834667,-110.934663,2335.249800,-0.012,-0.029,0.192
4,1010222,45.834608,-110.934772,2341.294466,-0.009,-0.029,0.192
...,...,...,...,...,...,...,...
558,1010225,45.835393,-110.933286,2297.427003,-0.009,-0.018,0.193
559,1010225,45.835329,-110.933493,2297.502624,0.000,-0.033,0.192
560,1010225,45.835263,-110.933696,2297.471755,0.004,-0.018,0.194
561,1010225,45.835197,-110.933897,2297.452760,0.004,-0.026,0.193


### Now, we need to call the original photo EXIF information to adjust pitch, roll, and yaw. 

In [24]:
# Import exiftoolhelper from the package exiftool
from exiftool import ExifToolHelper

In [25]:
# Get the unique value for the flights and add the folder name to a list
imagery_folders = RTKLIB['flight'].unique().tolist()
imagery_folders

['101_0222', '101_0223', '101_0224', '101_0225']

In [26]:
# Create an empty list to hold the imagery names
imagery_files = []
# for each path in the imagery folders of interest, read in the .jpgs
for path in imagery_folders:
    # Append the empty list to create a list of lists
    imagery_files.append(glob.glob(imagery_path + path + '/*.JPG'))

In [27]:
# Create a loop to turn the list of lists generated in the above code to a flat list
imagery_flat_list = []
# For each sublist in the above list of lists, append the item of that list to a singular list
for sublist in imagery_files:
    for item in sublist:
        imagery_flat_list.append(item)

In [28]:
# Create an empty list to hold the exif_data
exif_data = []
# With the ExifToolHelper, get tag information for each image
with ExifToolHelper() as et:
    # Specify what tag information you would like to capture
    for d in et.get_tags(imagery_flat_list, tags = ['Pitch','Roll','Yaw']):
        # append the list 
        exif_data.append(d)
        # Create a dataframe from the list values 
        exif_df = pd.DataFrame.from_dict(exif_data)

In [29]:
exif_df = exif_df.sort_values(by = 'SourceFile')
exif_df.reset_index(drop = True)
exif_df

,SourceFile,MakerNotes:Pitch,MakerNotes:Roll,MakerNotes:Yaw
20,Imagery/101_0222/101_0222_0001.JPG,-11.8,-4.6,-125.500000
32,Imagery/101_0222/101_0222_0002.JPG,-3.0,-2.5,-128.000000
37,Imagery/101_0222/101_0222_0003.JPG,-3.6,-2.7,-128.199997
9,Imagery/101_0222/101_0222_0004.JPG,-1.0,-2.2,-127.699997
18,Imagery/101_0222/101_0222_0005.JPG,-1.8,-3.0,-127.599998
...,...,...,...,...
485,Imagery/101_0225/101_0225_0156.JPG,-5.1,0.5,-115.699997
484,Imagery/101_0225/101_0225_0157.JPG,-1.9,-4.3,-114.900002
466,Imagery/101_0225/101_0225_0158.JPG,-5.8,-4.4,-114.400002
467,Imagery/101_0225/101_0225_0159.JPG,-4.1,-5.0,-114.300003


In [30]:
file_names = []
for f in exif_df['SourceFile']:
    name = os.path.basename(f)
    file_names.append(name)
exif_df['file_names'] = file_names
exif_df

,SourceFile,MakerNotes:Pitch,MakerNotes:Roll,MakerNotes:Yaw,file_names
20,Imagery/101_0222/101_0222_0001.JPG,-11.8,-4.6,-125.500000,101_0222_0001.JPG
32,Imagery/101_0222/101_0222_0002.JPG,-3.0,-2.5,-128.000000,101_0222_0002.JPG
37,Imagery/101_0222/101_0222_0003.JPG,-3.6,-2.7,-128.199997,101_0222_0003.JPG
9,Imagery/101_0222/101_0222_0004.JPG,-1.0,-2.2,-127.699997,101_0222_0004.JPG
18,Imagery/101_0222/101_0222_0005.JPG,-1.8,-3.0,-127.599998,101_0222_0005.JPG
...,...,...,...,...,...
485,Imagery/101_0225/101_0225_0156.JPG,-5.1,0.5,-115.699997,101_0225_0156.JPG
484,Imagery/101_0225/101_0225_0157.JPG,-1.9,-4.3,-114.900002,101_0225_0157.JPG
466,Imagery/101_0225/101_0225_0158.JPG,-5.8,-4.4,-114.400002,101_0225_0158.JPG
467,Imagery/101_0225/101_0225_0159.JPG,-4.1,-5.0,-114.300003,101_0225_0159.JPG


In [31]:
final_export = location_export.join(exif_df)

In [32]:
final_export = final_export.drop(columns = ['SourceFile','FlightID'])
final_export = final_export.rename(columns = {'MakerNotes:Pitch':'Pitch','MakerNotes:Roll':'Roll','MakerNotes:Yaw':'Yaw'})
final_export = final_export.set_index('file_names')

In [33]:
final_export = final_export.sort_values(by = 'file_names')
final_export

,New_Lat,New_Lon,New_El,Lat_diff_m,Lon_diff_m,El_diff_m,Pitch,Roll,Yaw
file_names,,,,,,,,,
101_0222_0001.JPG,45.833814,-110.936241,2457.334347,-0.012,-0.018,0.193,-11.8,-4.6,-125.500000
101_0222_0002.JPG,45.833281,-110.937229,2530.114164,0.000,-0.005,0.195,-3.0,-2.5,-128.000000
101_0222_0003.JPG,45.833011,-110.937726,2554.384396,-0.002,-0.002,0.195,-3.6,-2.7,-128.199997
101_0222_0004.JPG,45.834309,-110.935324,2371.536688,-0.014,-0.028,0.192,-1.0,-2.2,-127.699997
101_0222_0005.JPG,45.833931,-110.936024,2444.436385,-0.013,-0.015,0.194,-1.8,-3.0,-127.599998
...,...,...,...,...,...,...,...,...,...
101_0225_0156.JPG,45.837183,-110.933522,2297.334239,-0.002,-0.027,0.193,-5.1,0.5,-115.699997
101_0225_0157.JPG,45.837251,-110.933305,2297.304161,-0.032,0.047,0.186,-1.9,-4.3,-114.900002
101_0225_0158.JPG,45.837000,-110.934798,2297.417129,0.038,-0.014,0.190,-5.8,-4.4,-114.400002


## Final Step: Adjust the elevation data to account for differences in Snow Depth between flight dates

In [34]:
# Read in the .csv file with all of the RTK Heights for the season
rtk_height = pd.read_csv('/Users/f67f911/Desktop/HG_PPK/RTK_Base_Height.csv')
# View the file
rtk_height

,Date,Snow Depth (cm),Height to RTK (cm),Corrected Height
0,20220212,95,220,40
1,20220218,140,245,65
2,20220325,159,253,73
3,20220505,170,270,90
4,20220510,179,261,81
5,20220517,82,168,-12


In [ ]:
# If the date you are processing for is in the rtk_height file, do the next block of code. 
# If not, skip to the next 
# Capture the value for the height diffences for the date of interest
diff_height = rtk_height[rtk_height['Date'] == int(date)]
# Call to make sure it was captured correctly
# If no data is displayed, it may be because no RTK Height information was captured that day
diff_height

In [ ]:
# Calculate the corrected elevation
corrected_el = (final_export['New_El'] - int(diff_height['Corrected Height'])/100)
# Populate the new elevation with the corrected el
final_export['New_El'] = corrected_el

In [35]:
# Run this code if no corrected height is provided
corrected_el = final_export['New_El']

In [36]:
# View the final export and if it looks correct, proceed to download!
final_export

,New_Lat,New_Lon,New_El,Lat_diff_m,Lon_diff_m,El_diff_m,Pitch,Roll,Yaw
file_names,,,,,,,,,
101_0222_0001.JPG,45.833814,-110.936241,2457.334347,-0.012,-0.018,0.193,-11.8,-4.6,-125.500000
101_0222_0002.JPG,45.833281,-110.937229,2530.114164,0.000,-0.005,0.195,-3.0,-2.5,-128.000000
101_0222_0003.JPG,45.833011,-110.937726,2554.384396,-0.002,-0.002,0.195,-3.6,-2.7,-128.199997
101_0222_0004.JPG,45.834309,-110.935324,2371.536688,-0.014,-0.028,0.192,-1.0,-2.2,-127.699997
101_0222_0005.JPG,45.833931,-110.936024,2444.436385,-0.013,-0.015,0.194,-1.8,-3.0,-127.599998
...,...,...,...,...,...,...,...,...,...
101_0225_0156.JPG,45.837183,-110.933522,2297.334239,-0.002,-0.027,0.193,-5.1,0.5,-115.699997
101_0225_0157.JPG,45.837251,-110.933305,2297.304161,-0.032,0.047,0.186,-1.9,-4.3,-114.900002
101_0225_0158.JPG,45.837000,-110.934798,2297.417129,0.038,-0.014,0.190,-5.8,-4.4,-114.400002


In [37]:
final_export.to_csv(date + 'Corrected_Imagery_Locs.csv')